In [ ]:
import json
import torch
import torch.nn.functional as F
import time
import random
import numpy as np
from transformers import RobertaTokenizer, RobertaForMultipleChoice
# from transformers import BertTokenizer, BertForMultipleChoice


In [ ]:
# SET RANDOM SEED
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# # Modelo y tokenizer
# bert = "google-bert/bert-base-multilingual-cased"
# tokenizer = BertTokenizer.from_pretrained(bert)
# model = BertForMultipleChoice.from_pretrained(bert)

roberta = "PlanTL-GOB-ES/roberta-base-bne"
tokenizer = RobertaTokenizer.from_pretrained(roberta)
model = RobertaForMultipleChoice.from_pretrained(roberta)

In [ ]:
# Cargar dataset
with open("1-test.json", "r", encoding="utf-8") as f:
    test_dataset = json.load(f)["data"]

def format_pair(context, question, choice):
    return f"{context} Pregunta: {question} Opción: {choice}"

def prepare_inputs(context, question, choices):
    paired_texts = [format_pair(context, question, choice) for choice in choices]
    #Tokenizamos el prompt --> se pasa a numeros
    encoded = tokenizer(paired_texts, padding=True, truncation=True, return_tensors="pt", max_length=1024)
    return {
        #IDs de los tokens del modelo
        'input_ids': encoded['input_ids'].unsqueeze(0),
        #máscara que indica qué tokens son reales y cuales son padding
        'attention_mask': encoded['attention_mask'].unsqueeze(0),
    }

#El modelo predice la opción correcta
def predict_answer(inputs, choices):
#torch.no_grad --> más rápido y consume menos memoria
  with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits.squeeze(0)  # [num_choices]
    #Convertimos los logits en probabilidades
    probs = F.softmax(logits, dim=-1)
    #Se selecciona el índice con más probabilidad
    predicted_index = torch.argmax(probs).item()

    #Verificamos que el modelo no haya devuelto un índice fuera del rango de opciones
    if predicted_index >= len(choices):
        print(f"Error: Índice de predicción {predicted_index} fuera de rango.")
        return None

    return choices[predicted_index]


In [ ]:
max_labels = max(len(item["choices"]) for item in json.load(open("1-test.json"))["data"])
correct_predictionsBERT = 0
total_questions = len(test_dataset)
predictions = []

start_time = time.time()
for item in test_dataset:
  is_correct_prediction = False
  context = item["context"]
  question = item["question"]
  choices = [choice["text"] for choice in item["choices"]]
  correct_answer = next(choice["text"] for choice in item["choices"] if choice["type"] == "correct answer")

  num_choices = len(choices)
  if num_choices > max_labels:
    print(f"Error: Pregunta con {num_choices} opciones. Supera el máximo permitido: {max_labels}")
    continue
  if num_choices == 0:
    print(f"Pregunta con {num_choices} opciones.")
    continue

  # prompt = format_prompt(context, question, choices)
  # inputsBERT = tokenizer_bert(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
  # predicted_answerBERT = predict_answerBERT(inputsBERT, choices, num_choices)
  inputs = prepare_inputs(context, question, choices)
  predicted_answer = predict_answer(inputs, choices)


  if correct_answer == predicted_answer:
    correct_predictionsBERT += 1
    is_correct_prediction = True


  predictions.append({
    "context": context,
    "question": question,
    "choices": choices,
    "correct_answer": correct_answer,
    "predicted_answer": predicted_answer,
    "is_correct": is_correct_prediction
  })


end_time = time.time()
elapsed_time = end_time - start_time

with open("predictionsBERT.json", "w", encoding="utf-8") as f:
    json.dump(predictions, f, indent=4, ensure_ascii=False)

print(f"Tiempo de ejecución: {elapsed_time:.2f} segundos")
print()
print(f"Total de preguntas evaluadas: {total_questions}")
print(f"Respuestas correctas BERT: {correct_predictionsBERT}")
print(f"Precisión del modelo BERT: {correct_predictionsBERT / total_questions:.2%}")
print()

RuntimeError: The expanded size of the tensor (633) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [5, 633].  Tensor sizes: [1, 514]